In [1]:
import pandas as pd
from sklearn.externals import joblib
import pycrfsuite
import nltk
train_file = "Trained NER Models/train_BIO_Hotels.pkl"
test_file = "Trained NER Models/test_BIO_Hotels.pkl"

In [3]:
def removeBIO_and_merge(y_values):
    all_tags = []
    for values_ in y_values:
        set_values = set()
        for value in values_:
            value = int(value[2:])
            set_values.add(value)
        all_tags.append(list(set_values))
        
    return all_tags

In [5]:
X_train_file = joblib.load(train_file)
X_test_file = joblib.load(test_file)

In [6]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]    

In [7]:
X_train = [sent2features(s) for s in X_train_file]
y_train = [sent2labels(s) for s in X_train_file]

X_test = [sent2features(s) for s in X_test_file]
y_test = [sent2labels(s) for s in X_test_file]

In [8]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [9]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})


In [10]:
trainer.train('hotels.crfsuite')

In [11]:
def filter_eos(a):
    a = filter(lambda a: a != "," and a != "." and a != "?" and a != "!", a)
    return a
def pos_tagging(sent):
    return nltk.pos_tag(filter_eos(nltk.word_tokenize(sent)))

In [12]:
tagger = pycrfsuite.Tagger()
tagger.open('hotels.crfsuite')
y_pred = [tagger.tag(xseq) for xseq in X_test]

In [13]:
import pdb
def removeBIO_and_merge(y_values):
    all_labels = []
    pdb.set_trace()
    for values_ in y_values:
        set_values = set()
        for value in values_:
            try:
                if value != 'O':
                    value = int(value[2:])
                    set_values.add(value)
            except:
                
                pdb.set_trace()
        all_labels.append(list(set_values))
        
    return all_labels

In [30]:
y_p = [tagger.tag(xseq) for xseq in X_test]

In [31]:
y_p[5]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'I-12',
 'I-12',
 'I-12',
 'I-12',
 'I-12',
 'I-12',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O'

In [32]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y1 = mlb.fit_transform(removeBIO_and_merge(y_test))
y2 = mlb.transform(removeBIO_and_merge(y_p))
print classification_report(y1,y2)

> <ipython-input-13-f47af1855af1>(5)removeBIO_and_merge()
-> for values_ in y_values:
(Pdb) c
> <ipython-input-13-f47af1855af1>(5)removeBIO_and_merge()
-> for values_ in y_values:
(Pdb) c
             precision    recall  f1-score   support

          0       0.90      0.21      0.34       292
          1       0.93      0.49      0.65       293
          2       0.00      0.00      0.00       187
          3       1.00      0.01      0.01       185
          4       0.82      0.06      0.11       148
          5       0.00      0.00      0.00       142
          6       1.00      0.03      0.07        88
          7       0.00      0.00      0.00        69
          8       0.80      0.05      0.09        85
          9       1.00      0.03      0.06        67
         10       0.00      0.00      0.00        92
         11       0.00      0.00      0.00        41
         12       0.29      0.03      0.05        71
         13       0.00      0.00      0.00        76
         14     

In [33]:
with open('predicted_data.txt','wb') as f:
    for value in X_test_file:
        print >> f, value 
f.close()

In [52]:
# y1 = mlb.inverse_transform(y1)
# y2 = mlb.inverse_transform(y2)
with open('predicted.txt','wb') as f:
    for value in xrange(len(y_p)):
        for ind in xrange(len(y_p[value])):
            print >> f, str(X_test_file[value][ind]),'-',str(y_p[value][ind]),
        print >> f,'\t', str(y2[value]), '\t',str(y1[value])
f.close()